In [2]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 \
#     evaluate==0.4.0 \
#     rouge_score==0.1.2 \
#     loralib==0.1.1 \
#     peft==0.3.0 --quiet

In [1]:
!nvidia-smi

Tue Aug 29 06:32:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:0C:00.0 Off |                  Off |
| 30%   34C    P2    65W / 300W |   2481MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:0D:00.0 Off |                  Off |
| 30%   

In [1]:
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM, 
                          AutoTokenizer, 
                          GenerationConfig, 
                          TrainingArguments, 
                          Trainer)
import torch
import time
import os
import evaluate
import pandas as pd
import numpy as np
from math import ceil

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-09-03 13:23:19.573288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 13:23:21.056797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-03 13:23:21.056953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  
torch.cuda.device_count()

1

In [3]:
from peft import PeftModel

def load_base_model(model_path="google/flan-t5-base"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path, torch_dtype=torch.bfloat16
    ).to(device)

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def load_from_peft_adapter(
    base_model_path, peft_model_path, train=False, merge_adapter=True
):
    model, tokenizer = load_base_model(base_model_path)
    model = PeftModel.from_pretrained(
        model, peft_model_path, torch_dtype=torch.bfloat16, is_trainable=train
    ).to(device)

    if merge_adapter:
        model = model.merge_and_unload()

        if train:
            for param in model.parameters():
                param.requires_grad = True

    # merge the adapter to the main model
    return model, tokenizer

def save_peft_adapter(model, model_path):
    model.save_pretrained(model_path)

def merge_peft_and_save(model, model_path):
    model = model.merge_and_unload()
    model.save_pretrained(model_path)

def save_tokenizer(tokenizer):
    tokenizer.save_pretrained(model_path)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [4]:
# class PeftModel:
#     @staticmethod
#     def load_base_model(model_path="google/flan-t5-base"):
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         model = AutoModelForSeq2SeqLM.from_pretrained(
#             model_path, torch_dtype=torch.bfloat16
#         ).to(device)
#         tokenizer = AutoTokenizer.from_pretrained(model_path)
#         return model, tokenizer

#     @staticmethod
#     def load_from_peft_adapter(
#         base_model_path, peft_model_path, train=False, merge_adapter=True
#     ):
#         model, tokenizer = self.load_base_model(base_model_path)
#         model = PeftModel.from_pretrained(
#             model, peft_model_path, torch_dtype=torch.bfloat16, is_trainable=train
#         ).to(device)

#         if merge_adapter:
#             model = model.merge_and_unload()

#             if train:
#                 for param in model.parameters():
#                     param.requires_grad = True

#         # merge the adapter to the main model
#         return model, tokenizer

#     @staticmethod
#     def save_peft_adapter(model, model_path):
#         model.save_pretrained(model_path)

#     @staticmethod
#     def merge_peft_and_save(model, model_path):
#         model = model.merge_and_unload()
#         model.save_pretrained(model_path)
        
#     @staticmethod
#     def save_tokenizer(tokenizer):
#         tokenizer.save_pretrained(model_path)

In [5]:
# # old load original model
# name='google/flan-t5-base'
# model, tokenizer = PeftModel.load_base_model(model_path=name)

In [6]:
# load original model
name='google/flan-t5-base'
model, tokenizer = load_base_model(model_path=name)

<a name='1.2'></a>
### 1.2 - Load Dataset and LLM

In [7]:
# load and aggregate raw data
import os
import json

# Specify the folder path containing the JSON files
folder_path = './data'

# Initialize an empty list to aggregate the data
data = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json') and "shorts" in filename:
        print(filename)
        file_path = os.path.join(folder_path, filename)
        
        # Read and parse JSON data from the file
        with open(file_path, 'r') as json_file:
            file_data = json.load(json_file)
            
            # Assuming each JSON file contains a list of dictionaries
            if isinstance(file_data, list):
                data.extend(file_data)

inshorts_scraped.json


In [8]:
data[0]

{'title': 'Man accused of sexually assaulting daughter granted bail by HC amid matrimonial dispute',
 'summary': 'Delhi HC granted bail to a man accused of sexually assaulting his daughter, noting that it cannot shut its eyes to matrimonial dispute between her parents and his false implication by "tutoring" cannot be ruled out. It observed she has been residing with the mother for over four years. The court also noted there was inordinate delay in FIR registration.',
 'link': 'https://www.outlookindia.com/national/delhi-hc-grants-bail-to-man-accused-of-sexually-assaulting-daughter-news-311110/amp?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts',
 'image_link': 'https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2023/08_aug/16_wed/img_1692206351958_642.jpg?',
 'source': 'inshorts',
 'full_text': 'Delhi HC Grants Bail To Man Accused Of Sexually Assaulting Daughter Justice Vikas Mahajan observed the girl has been residing with the mother for more than 4 years a

# curate data

In [9]:
import random

data = [news for news in data if news["full_text"] != "" and "JavaScript is not available" not in news["full_text"] and "reuters" not in news["link"]]
random.shuffle(data)
len(data)

22477

In [10]:
import re

for news in data:
    if "<p>" in news["summary"]:
        # Regular expression to match content between <p> tags
        pattern = re.compile(r'<p>(.*?)</p>', re.DOTALL)
        matches = pattern.findall(news["summary"])

        # Extracted content from <p> tags
        extracted_content = [re.sub(r'<.*?>', '', match) for match in matches]
        news["summary"] = max(extracted_content, key=len)

# Dataset

In [11]:
import random
from tqdm import tqdm
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split


class TextDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.inputs = []
        self.labels = []
        
        for news in data:   
            # input_prompt, label = self._get_summary_prompt(news)
            # self.inputs.append(input_prompt)
            # self.labels.append(label)
            
            input_prompt, label = self._get_title_prompt(news)
            self.inputs.append(input_prompt)
            self.labels.append(label)
            
        """
        Combine the lists using zip
        Shuffle the combined list
        Unpack the shuffled pairs back into separate lists
        And then tokenize
        """
        combined = list(zip(self.inputs, self.labels))
        random.shuffle(combined)
        self.inputs, self.labels = zip(*combined)

        # tokenize
        self.inputs = tokenizer(self.inputs, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt").input_ids

        self.labels = tokenizer(self.labels, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt").input_ids
            
    def __len__(self): 
        return len(self.inputs)

    def __getitem__(self, idx): 
        return self.inputs[idx], self.labels[idx]
    
    @staticmethod
    def _get_summary_prompt(example):
        # word count round off
        multiple = 25
        word_count = len(example["summary"].split())
        word_count = int(round(word_count / multiple)) * multiple
        
        start_prompt = f'Summarize this news article in {word_count} words.\n\n'
        end_prompt = '\n\nSummary: '

        prompt = start_prompt + example["full_text"] + end_prompt

        return prompt, example["summary"]
    
    @staticmethod
    def _get_title_prompt(example):
        # word count round off
        multiple = 5
        word_count = len(example["title"].split())
        word_count = int(ceil(word_count / multiple)) * multiple
        
        start_prompt = f'Give a title to the given news article in not more than {word_count} words.\n\n'
        mid_prompt = '\n\nSummary: '
        end_prompt = '\n\nTitle: '

        prompt = start_prompt + example["full_text"] + mid_prompt + example["summary"] + end_prompt
        return prompt, example["title"]

In [12]:
train_data = TextDataset(data, tokenizer)
# test_data = TextDataset(data, tokenizer)

In [13]:
train_data[0]

(tensor([ 6434,     3,     9,  2233,    12,     8,   787,  1506,  1108,    16,
            59,    72,   145,   627,  1234,     5,  1547,    43,  2031,  3030,
            12,  3197,    80,   223,    16,     8,   874,    18, 19515,   332,
          1755,   196,   939,   227,   352,     3,   632,  4949,   323,   778,
            16,     8,  2101,   192,  1031,     5,   461,  2818,     6,    44,
             8,   337,  5669,   116,    79,  8151,    70,   511,  2541,  1453,
             6,  1547,  3030,    12,  4943,    46,  8943,   547,   447,  2391,
            18,  5981,    15,    17,  1369,   213,  3705,    63, 16296,  1635,
          4701,    26,     9,   208,     3, 25202,    15,    26,   223,    12,
           607,    16,   869,    12,   428,     3,     9, 15937,    13,   112,
            20,   900, 11026,   200,    16,     8,  1910,     5,   978, 22716,
          7673,    13,     3,  4591,  3154,   326,  8537, 11607,  2139,  1547,
         15389,   323,     8,  2387,    13, 11321,  

In [14]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        
        if param.requires_grad:
            trainable_model_params += param.numel()
    
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [15]:
# # FULL MODEL TRAINING
# EPOCH = 1

# training_args = TrainingArguments(
#                                   save_steps=5000,
#                                   warmup_steps=10,
#                                   logging_steps=100,
#                                   weight_decay=0.01,
#                                   num_train_epochs=EPOCH,
#                                   logging_dir='./logs',
#                                   output_dir='./checkpoint',
#                                   per_device_eval_batch_size=32,
#                                   per_device_train_batch_size=32)

# Trainer(model=model,
#         args=training_args,
#         eval_dataset=test_data,
#         train_dataset=train_data,
#         data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 
#                                     'labels': torch.stack([f[1] for f in data])}).train()

In [16]:
# PEFT MODEL TRAINING
from peft import LoraConfig, get_peft_model, TaskType


EPOCH = 10

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model, lora_config)
# peft_model = torch.nn.DataParallel(peft_model)
print(print_number_of_trainable_model_parameters(peft_model))


peft_training_args = TrainingArguments(
                                  # save_steps=5000,
                                  save_strategy="no",
                                  warmup_steps=10,
                                  logging_steps=1000,
                                  weight_decay=0.01,
                                  num_train_epochs=EPOCH,
                                  logging_dir='./logs',
                                  output_dir='./checkpoint',
                                  learning_rate=0.0001,
                                  auto_find_batch_size=True)
    
peft_trainer = Trainer(
                model=peft_model,
                args=peft_training_args,
                train_dataset=train_data,
                data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]), 
                                            'labels': torch.stack([f[1] for f in data])})

peft_trainer.train()

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,4.132800
2000,0.137800
3000,0.113200
4000,0.104100
5000,0.099900
6000,0.096900
7000,0.093100
8000,0.091000
9000,0.089200
10000,0.088000


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=28100, training_loss=0.23320891139346087, metrics={'train_runtime': 9754.7113, 'train_samples_per_second': 23.042, 'train_steps_per_second': 2.881, 'total_flos': 1.5635652772626432e+17, 'train_loss': 0.23320891139346087, 'epoch': 10.0})

# save model

In [17]:
# # old save peft adapter
# adapter_path = "./checkpoint/adapter/"
# PeftModel.save_peft_adapter(model=peft_model, model_path=adapter_path)

In [18]:
# # old merge peft with main model
# # and save the model
# model_path = "./checkpoint/"
# PeftModel.merge_peft_and_save(model=peft_model, model_path=model_path)

In [17]:
# save peft adapter
adapter_path = "./checkpoint/title_adapter/"
save_peft_adapter(model=peft_model, model_path=adapter_path)

In [ ]:
# merge peft with main model
# and save the model
model_path = "./checkpoint/"
merge_peft_and_save(model=peft_model, model_path=model_path)

# load pretrained flan t5 

In [68]:
# # old load original model
# model_name='google/flan-t5-base'
# original_model, original_tokenizer = PeftModel.load_base_model(model_path=model_name)

In [ ]:
# load original model
model_name='google/flan-t5-base'
original_model, original_tokenizer = load_base_model(model_path=model_name)

# load the saved peft inshorts model

In [21]:
# model_path = "./checkpoint/"
# peft_model, tokenizer = PeftModel.load_base_model(model_path=model_path)

In [ ]:
model_path = "./checkpoint/adapter/"
peft_model, tokenizer = load_from_peft_adapter(model_name, model_path)

# infer

In [70]:
dash_line = '-'.join('' for x in range(100))

index = 1000
news = data[index]
full_text = news['full_text']
baseline_human_summary = news['summary']

# word count round off
multiple = 25
word_count = len(baseline_human_summary.split())
word_count = int(round(word_count / multiple)) * multiple

start_prompt = f'Summarize this news article in {word_count} words.\n\n'
end_prompt = '\n\nSummary: '

prompt = start_prompt + full_text + end_prompt

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=4))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'FULL TEXT:\n{full_text}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


# EVALUATE
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=[original_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=[peft_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('INSTRUCT MODEL:')
print(instruct_model_results)

---------------------------------------------------------------------------------------------------
FULL TEXT:
Please enable Javascript to watch this video 
  
 Grainy, poor-quality surveillance footage showing the slow gait of a large cat-like animal walking along a street in densely populated Norwalk has created a mystery. 
  
 At first, the person who saw the animal early Tuesday thought it was a mountain lion, but that theory has been discounted by state wildlife officials, City Manager Jeff Hobbs said Friday afternoon. 
  
 The California Department of Fish and Wildlife's "undisputed expert" determined the animal shown in the video is not a cougar, Hobbs wrote in an email headlined "'That's No Mountain Lion,' Says CA Dept Fish & Wildlife Expert." 
  
 "Department officials still cannot definitively identify the type of animal. They will continue to investigate," Hobbs wrote. 
  
 A mountain lion sighting was reported about 3:45 a.m. in the 11000 block of Tina Street (map), accordi

In [72]:
dash_line = '-'.join('' for x in range(100))

index = 5670
news = data[index]
full_text = news['full_text']
baseline_human_summary = news['title']

# word count round off
multiple = 5
word_count = len(news["title"].split())
word_count = int(ceil(word_count / multiple)) * multiple

start_prompt = f'Give a title to the given news article in not more than {word_count} words.\n\n'
mid_prompt = '\n\nSummary: '
end_prompt = '\n\nTitle: '

prompt = start_prompt + news["full_text"] + mid_prompt + news["summary"] + end_prompt

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'FULL TEXT:\n{full_text}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


# EVALUATE
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=[original_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=[peft_model_text_output],
    references=[baseline_human_summary],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('INSTRUCT MODEL:')
print(instruct_model_results)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1508048/1478766500.py:6 in <cell line: 6>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1508048/1478766500.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'title'